In [1]:
from pathlib import Path
import sys
root = Path.cwd().parent
sys.path.append(str(root))

import urbans
from grammar.dictionary import dic_en2jp
from grammar.source_grammar import EN_source_transduction_grammar as src_grammar
from grammar.transduction_rule import src_to_target_grammar

from grammar.postproc import post_processing, pref_pattern

In [2]:
post_processor = post_processing()

In [3]:
# The translator in revised urbans can be created with a empty dictionary and acting as a parser.
word_parser = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dict())

In [4]:
with open('Questions.txt') as f:
    lines = f.readlines()

lines = list(set(lines))
del lines[lines.index('\n')]

## Word Parsing

In [113]:
tag_word_set, failed_sentences, ambiguity_sentences = word_parser.parse_words(sentences=lines)

100%|██████████| 1000/1000 [00:14<00:00, 69.85it/s]

Word parsing completed! 0 sentences failed. 356 sentences occurred ambiguity.


In [6]:
# reparsing
tag_word_set_2, failed_sentences_2, ambiguity_sentences_2 = word_parser.parse_words(sentences=['Was a Chinese actor that M2 was written by and starred a composer\n'])

Word parsing completed! 0 sentences failed. 1 sentences occurred ambiguity.


In [15]:
tag_word_set.keys()

dict_keys(['did', 'Name', 'V', 'det', 'role', 'of', 'commonNoun', 'pS', 'AdjNa', 'F', 'VT', 'conjN', 'conj', 'punct', 'R', 'VMod', 'P', 'punctN', 'VsrT', 'by', 'VsrMod', 'was', 'whose', 'VPassOrgMod', 'Vsr', 'VPassOrgT', 'VOrgPassMod', 'Cnt', 'nat', 'VOrgPassT', 'detSub', 'Adj', 'WhW', 'NPQ'])

In [ ]:
import nltk
analyze_sen = "Who was influenced by a composer influenced by M3 and influenced by M4 and M5 and influenced by M1"
for i in ambiguity_sentences[analyze_sen]:
    i.draw()

## Translation

In [8]:
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)

In [31]:
trans_sentence, trans_map = e2j_translator.translate(ambiguity_sentences, remove_space=True)

100%|██████████| 19823/19823 [05:08<00:00, 64.32it/s]


In [36]:
len(trans_map)

784

#### Disambiguity
784 out of 19823 ambiguities harmful

In [79]:
# with open('ambiguinous_question.txt', 'w') as f:
#     for line in ambiguity_sentences:
#         f.write(line)
try: 
    ambiguity_sentences
except NameError:
    ambiguity_sentences = None


if ambiguity_sentences == None:
    with open('ambiguinous_question.txt', 'r') as f:
        ambiguity_sentences = f.readlines()


In [87]:
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_sentence, trans_map = e2j_translator.translate(ambiguity_sentences, remove_space=True, prefered_pattern=pref_pattern)

100%|██████████| 37299/37299 [08:02<00:00, 77.35it/s] 



536 >> 438 >> 388 >> 377 out of 17648

In [79]:
test_sent =  'Who was influenced by a composer influenced by M3 and influenced by M4 and M5 and influenced by M1'
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_sentence, trans_map = e2j_translator.translate(test_sent, remove_space=True, prefered_pattern=pref_pattern)

100%|██████████| 1/1 [00:00<00:00, 18.91it/s]


## Test/Evaluation

In [7]:
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
vq = random.choices(lines, k=1000)

In [71]:
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_vq, trans_map_vq = e2j_translator.translate(vq, remove_space=True, prefered_pattern=pref_pattern)

100%|██████████| 1000/1000 [00:10<00:00, 91.12it/s]


In [66]:
vq_trans = [(s,t) for s, t in zip(vq, trans_vq)]

### Evaluate with GOLD

In [51]:
gd_jp = pd.read_csv('CWQ Annotation - GOLD-JP.csv')

In [52]:
dev, test = train_test_split(gd_jp, test_size=0.66, random_state=42)

In [53]:
scr_dev = list(dev['questionPatternModEntities'])
scr_test = list(test['questionPatternModEntities'])
gd_dev = list(dev['questionPatternModEntitiesJP'])
gd_test = list(test['questionPatternModEntitiesJP'])

In [54]:
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_dev, trans_map_dev = e2j_translator.translate(scr_dev, remove_space=True, prefered_pattern=pref_pattern)
trans_dev = post_processor.replace(trans_dev)

100%|██████████| 52/52 [00:01<00:00, 51.40it/s]


In [55]:
from sacrebleu.metrics import BLEU
from sacrebleu.tokenizers import tokenizer_ja_mecab

In [56]:
bleu = BLEU(tokenize='ja-mecab')
bleu.corpus_score(hypotheses=trans_dev, references=[gd_dev])

BLEU = 97.98 99.3/98.4/97.5/96.9 (BP = 1.000 ratio = 1.000 hyp_len = 1206 ref_len = 1206)

On dev: BLEU = 97.98 99.3/98.4/97.5/96.9 (BP = 1.000 ratio = 1.000 hyp_len = 1206 ref_len = 1206)

In [57]:
comp = []
for a, b in zip(gd_dev,trans_dev):
    if a != b:
        comp.append((a, b))

In [60]:
pref_pattern = [{"S -> was NominalSub Vobl","commonNoun -> F commonNounHead"},
                {"S -> was NominalSub Vobl","commonNounHead -> F commonNounHead"},
                {"caseS -> Name pS"},
                {"NPQ -> WhWNominal"}
                ]
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_test, trans_map_test = e2j_translator.translate(scr_test, remove_space=True, prefered_pattern=pref_pattern)
trans_test = post_processing(trans_test,pattern_pairs=pattern_pairs)

100%|██████████| 103/103 [00:01<00:00, 75.56it/s]


In [61]:
bleu = BLEU(tokenize='ja-mecab')
bleu.corpus_score(hypotheses=trans_test, references=[gd_test])

BLEU = 97.07 99.2/97.9/96.4/95.0 (BP = 1.000 ratio = 1.000 hyp_len = 2306 ref_len = 2307)

On test: BLEU = 97.07 99.2/97.9/96.4/95.0 (BP = 1.000 ratio = 1.000 hyp_len = 2306 ref_len = 2307)

In [46]:
comp_test = []
for a, b in zip(gd_test,trans_test):
    if a != b:
        comp_test.append((a, b))

In [47]:
len(comp_test)

19

# Dataset translation

In [16]:
# load questions
with open('Questions.txt') as f:
    lines = f.readlines()

lines = list(set(lines))
del lines[lines.index('\n')]

In [18]:
pref_pattern = [{"S -> was NominalSub Vobl","commonNoun -> F commonNounHead"},
                {"S -> was NominalSub Vobl","commonNounHead -> F commonNounHead"},
                {"caseS -> Name pS"},
                {"NPQ -> WhWNominal"}
                ]
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
lines_jp, maps_jp = e2j_translator.translate(lines, remove_space=True, prefered_pattern=pref_pattern)
lines_jp = post_processor.replace(lines_jp)

100%|██████████| 1/1 [00:00<00:00, 95.11it/s]


In [67]:
len(lines) == len(lines_jp)
lines_en2jp = {}
for en, jp in zip(lines, lines_jp):
    lines_en2jp[en] = jp

In [70]:
# import csv

# with open('ambiguity_maps.csv', 'w') as f:
#     for key in maps_jp.keys():
#         f.write("%s, %s\n" % (key, maps_jp[key]))

# Translate samples for manual evaluation

In [8]:
me_en = pd.read_csv('CWQ Annotation - EN.csv')

In [9]:
sentences = list(me_en['questionWithBrackets'].dropna())

In [10]:
import re

In [27]:
# replace bracketed entities with M1 as placeholder
ent_register = []
rep_sentences = []
for seten in sentences:
    ents = re.findall("\[.*?\]", set)
    for ent in ents:
        ent_register.append(ent)
    rep_sentences.append(re.sub("\[.*?\]", 'M1', set))

In [28]:
e2j_translator = urbans.Translator(src_grammar=src_grammar, src_to_tgt_grammar=src_to_target_grammar, src_to_tgt_dictionary=dic_en2jp)
trans_rep_sen, trans_map_test = e2j_translator.translate(rep_sentences, remove_space=True, prefered_pattern=pref_pattern)
trans_sen = post_processor(trans_rep_sen)

100%|██████████| 42/42 [00:01<00:00, 25.44it/s]


In [13]:
# put the bracketed entities back
me_jp = []
for sen in trans_sen:
    while re.search('M1', sen):
        sen = sen.replace('M1', ent_register.pop(0), 1)
    me_jp.append(sen)


In [14]:
dic = [{'questionWithBrackets':en, 'questionWithBrackets_jp':jp} for en,jp in zip(sentences, me_jp)]

In [15]:
import csv

In [16]:
with open('CWQ Annotation - JP.csv','w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['questionWithBrackets', 'questionWithBrackets_jp'])
    writer.writeheader()
    writer.writerows(dic)

# Statistics

In [4]:
grammar = set(src_grammar.split('\n'))

In [5]:
grammar.remove('')

In [6]:
len(grammar)

252

252 totally, 37 terminals

In [7]:
symb = []
for g in grammar:
    for s in g.split():
        if "\'" in s or "\"" in s:
            continue
        symb.append(s)
symb = set(symb)



125 EN monolingual grammar, 22 terminals

dict terminal 75-122 pairs 287

In [61]:
list_en = set()
list_jp = set()
ct = 0
for tag, dct in dic_en2jp.items():
    for e, j in dct.items():
        list_en.add(e)
        list_jp.add(j)
        ct+=1